In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tqdm

from tqdm import tqdm

build_data = False

class DogsvsCats():
    
    img_size = 50
    cats = '/Users/varun/documents/deep_learning/Learn_PYTORCH/PetImages/Cat'
    dogs = '/Users/varun/documents/deep_learning/Learn_PYTORCH/PetImages/Dog'
    labels = {cats:0,dogs:1}
    train_data = []
    catcount = 0
    dogcount = 0

    def make_training_data(self):

        for label in self.labels:

            for f in tqdm(os.listdir(label)):
                    try:
                        path = os.path.join(label,f)
                        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img,(img_size,img_size))
                        ids = np.eye(2)[self.labels[label]]
                        train_data.append([np.array(img),ids])

                        if label == self.cats:
                                self.catcount +=1
                        else:
                                self.dogcount +=1
                    except Exception as e:
                        pass

            np.random.shuffle(train_data)
            np.save("train_data.npy",train_data)

            print(f"DOGS:  {self.dogcount}")
            print(f"CATS:  {self.catcount}")

if build_data:
    dogsvscats = DogsvsCats()
    dogsvscats.make_training_data()
else:
    train_data = np.load("train_data.npy",allow_pickle = True)

In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F

training_data = train_data

class Net(nn.Module):
    def __init__(self):
        super().__init__() 
        self.conv1 = nn.Conv2d(1, 32, 5) 
        self.conv2 = nn.Conv2d(32, 64, 5) 

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
       
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)


net = Net()
print(net)


import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1  
val_size = int(len(X)*VAL_PCT)
print(val_size)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

BATCH_SIZE = 100
EPOCHS = 3

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]

        net.zero_grad()

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    

    print(f"Epoch: {epoch}. Loss: {loss}")

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


  0%|          | 0/228 [00:00<?, ?it/s]

2531


  0%|          | 0/228 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.23047776520252228


  0%|          | 0/228 [00:00<?, ?it/s]

Epoch: 1. Loss: 0.1692119538784027


100%|██████████| 228/228 [01:45<00:00,  2.16it/s]

Epoch: 2. Loss: 0.14403094351291656


In [72]:
BATCH_SIZE = 100
EPOCHS = 15

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): 
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]

        net.zero_grad()

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()   

    print(f"Epoch: {epoch}. Loss: {loss}")

  0%|          | 0/228 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.13259096443653107


  0%|          | 0/228 [00:00<?, ?it/s]

Epoch: 1. Loss: 0.12254723906517029


  0%|          | 0/228 [00:00<?, ?it/s]

...

Epoch: 12. Loss: 0.029677076265215874


  0%|          | 0/228 [00:00<?, ?it/s]

Epoch: 13. Loss: 0.05638907849788666


100%|██████████| 228/228 [01:41<00:00,  2.25it/s]

Epoch: 14. Loss: 0.06582918763160706


In [73]:
true = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            true += 1
        total += 1
print("Accuracy: ", round(true/total, 3)*100)

100%|██████████| 2531/2531 [00:05<00:00, 436.80it/s]

Accuracy:  76.5
